<a href="https://colab.research.google.com/github/naenae4796/copilot-codespaces-vscode/blob/main/Problem_Solving_w_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import heapq
import random
import time
from collections import deque

# Puzzle Setup and Utilities
def goal_state(size):
    return tuple(list(range(1, size * size)) + [0])

def print_state(state, size):
    for i in range(size):
        print(state[i * size:(i + 1) * size])
    print()

def get_neighbors(state, size):
    neighbors = []
    zero_index = state.index(0)
    row, col = divmod(zero_index, size)
    moves = []
    if row > 0: moves.append(-size)      # Up
    if row < size - 1: moves.append(size)  # Down
    if col > 0: moves.append(-1)         # Left
    if col < size - 1: moves.append(1)   # Right

    for move in moves:
        new_index = zero_index + move
        new_state = list(state)
        new_state[zero_index], new_state[new_index] = new_state[new_index], new_state[zero_index]
        neighbors.append(tuple(new_state))
    return neighbors

def random_walk(start_state, steps, size):
    state = start_state
    for _ in range(steps):
        neighbors = get_neighbors(state, size)
        state = random.choice(neighbors)
    return state

def reconstruct_path(came_from, start, goal):
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start)
    path.reverse()
    return path

# Heuristics
def misplaced_tiles(state, goal):
    return sum(1 for i in range(len(state)) if state[i] != 0 and state[i] != goal[i])

def manhattan_distance(state, goal, size):
    distance = 0
    for i, tile in enumerate(state):
        if tile == 0:
            continue
        goal_index = goal.index(tile)
        row1, col1 = divmod(i, size)
        row2, col2 = divmod(goal_index, size)
        distance += abs(row1 - row2) + abs(col1 - col2)
    return distance

# Search Algorithms
def bfs(start, goal, size):
    frontier = deque([start])
    came_from = {start: None}
    visited = set([start])
    nodes_expanded = 0

    while frontier:
        current = frontier.popleft()
        nodes_expanded += 1
        if current == goal:
            return reconstruct_path(came_from, start, goal), nodes_expanded
        for neighbor in get_neighbors(current, size):
            if neighbor not in visited:
                frontier.append(neighbor)
                visited.add(neighbor)
                came_from[neighbor] = current
    return None, nodes_expanded

def astar(start, goal, size, heuristic):
    frontier = []
    heapq.heappush(frontier, (0, start))
    came_from = {start: None}
    cost_so_far = {start: 0}
    nodes_expanded = 0

    while frontier:
        _, current = heapq.heappop(frontier)
        nodes_expanded += 1
        if current == goal:
            return reconstruct_path(came_from, start, goal), nodes_expanded
        for neighbor in get_neighbors(current, size):
            new_cost = cost_so_far[current] + 1
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                priority = new_cost + heuristic(neighbor, goal, size)
                heapq.heappush(frontier, (priority, neighbor))
                came_from[neighbor] = current
    return None, nodes_expanded

# Experiment Runner
def run_experiments():
    walk_lengths = [5, 10, 20, 40, 80]
    sizes = [3, 4]

    for size in sizes:
        print(f"\n==== {size}x{size} Puzzle ====")
        goal = goal_state(size)
        for steps in walk_lengths:
            for i in range(3):
                start = random_walk(goal, steps, size)
                print(f"\n--- Random Walk {steps} Steps (Test {i+1}) ---")
                print("Start State:")
                print_state(start, size)

                # BFS
                start_time = time.time()
                path, expanded = bfs(start, goal, size)
                duration = time.time() - start_time
                print(f"BFS: Path Length = {len(path) - 1}, Nodes Expanded = {expanded}, Time = {duration:.3f}s")

                # A* Misplaced
                start_time = time.time()
                path, expanded = astar(start, goal, size, lambda s, g, sz: misplaced_tiles(s, g))
                duration = time.time() - start_time
                print(f"A* (Misplaced): Path Length = {len(path) - 1}, Nodes Expanded = {expanded}, Time = {duration:.3f}s")

                # A* Manhattan
                start_time = time.time()
                path, expanded = astar(start, goal, size, manhattan_distance)
                duration = time.time() - start_time
                print(f"A* (Manhattan): Path Length = {len(path) - 1}, Nodes Expanded = {expanded}, Time = {duration:.3f}s")

# Run all experiments
run_experiments()



==== 3x3 Puzzle ====

--- Random Walk 5 Steps (Test 1) ---
Start State:
(1, 0, 3)
(4, 2, 5)
(7, 8, 6)

BFS: Path Length = 3, Nodes Expanded = 15, Time = 0.000s
A* (Misplaced): Path Length = 3, Nodes Expanded = 4, Time = 0.000s
A* (Manhattan): Path Length = 3, Nodes Expanded = 4, Time = 0.000s

--- Random Walk 5 Steps (Test 2) ---
Start State:
(1, 2, 3)
(4, 5, 6)
(7, 0, 8)

BFS: Path Length = 1, Nodes Expanded = 4, Time = 0.000s
A* (Misplaced): Path Length = 1, Nodes Expanded = 2, Time = 0.000s
A* (Manhattan): Path Length = 1, Nodes Expanded = 2, Time = 0.000s

--- Random Walk 5 Steps (Test 3) ---
Start State:
(1, 2, 3)
(4, 5, 0)
(7, 8, 6)

BFS: Path Length = 1, Nodes Expanded = 3, Time = 0.000s
A* (Misplaced): Path Length = 1, Nodes Expanded = 2, Time = 0.000s
A* (Manhattan): Path Length = 1, Nodes Expanded = 2, Time = 0.000s

--- Random Walk 10 Steps (Test 1) ---
Start State:
(1, 2, 3)
(4, 0, 5)
(7, 8, 6)

BFS: Path Length = 2, Nodes Expanded = 13, Time = 0.000s
A* (Misplaced): Path 